In [ ]:
!pip install pyxlsb
from sklearn.metrics.pairwise import cosine_distances
import numpy as np
from google.colab import drive
import matplotlib.pyplot as plt
import pandas as pd
drive.mount('/content/drive')
df = pd.read_excel('/content/drive/My Drive/Virtual/stcRecomandations_T3.xlsb')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.user_id_maped.nunique()

11578

In [ ]:
df.program_name.nunique() #number of unigue value without concedration of deplucate

8013

In [ ]:
df.shape

(1048575, 6)

In [ ]:
df.rating.value_counts()

1    264293
4    262113
3    261505
2    260664
Name: rating, dtype: int64

In [ ]:
import re

def clean_title(title):
  return re.sub("[^a-zA-Z0-9 ]","", title)

In [ ]:
df['cleaned_title']=df['program_name'].apply(clean_title)
df

NameError: ignored

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
#here it convert the feature to number and  what itdoes it look to it twoo word at a time soo if we have (Reema Abdullah Alharbi) it will take it first Reema Abdullah and Abdulah Alharbi
vectorizer=TfidfVectorizer(ngram_range=(1,3))
tfidf= vectorizer.fit_transform(df['cleaned_title'])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search(title):
    title = clean_title(title)
    query_vec = vectorizer.transform([title])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -5)[-5:]
    results = df.iloc[indices].iloc[::-1]

    return results


In [ ]:
import ipywidgets as widgets
from IPython.display import display

movie_input = widgets.Text(
    description='Movie Title:',
    disabled=False
)
movie_list = widgets.Output()

def on_type(data):
    with movie_list:
        movie_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            display(search(title))

movie_input.observe(on_type, names='value')


display(movie_input, movie_list)


Text(value='', description='Movie Title:')

Output()

In [ ]:
program_dict = {}
program_id_counter = 1

for index, row in df.iterrows():
    program_name = row['program_name']

    if program_name in program_dict:
        program_id = program_dict[program_name]
    else:
        program_id = program_id_counter
        program_dict[program_name] = program_id
        program_id_counter += 1

    df.at[index, 'program_id'] = program_id



In [ ]:
df[(df["program_name"]=="100 treets")].head(3)

,Unnamed: 0,user_id_maped,program_name,rating,date_,program_genre,cleaned_title,program_id
0,0,26138,100 treets,1,2017-05-27,Drama,100 treets,1.0
154,154,3422,100 treets,2,2017-04-11,Drama,100 treets,1.0
1147,1147,15168,100 treets,1,2017-05-29,Drama,100 treets,1.0


In [ ]:
movie_id = 2

#def find_similar_movies(movie_id):
movie = df[df["program_id"] == movie_id]
movie

,Unnamed: 0,user_id_maped,program_name,rating,date_,program_genre,cleaned_title,program_id
1,1,7946,Moana,1,2017-05-21,Animation,Moana,2.0
18,18,26546,Moana,1,2017-06-25,Animation,Moana,2.0
19,19,20306,Moana,1,2017-06-23,Animation,Moana,2.0
48,48,28687,Moana,2,2017-05-30,Animation,Moana,2.0
191,191,16739,Moana,1,2017-05-27,Animation,Moana,2.0
...,...,...,...,...,...,...,...,...
1047660,1047660,22833,Moana,3,2017-06-03,Animation,Moana,2.0
1047786,1047786,32606,Moana,2,2017-05-10,Animation,Moana,2.0
1047833,1047833,23131,Moana,2,2017-05-13,Animation,Moana,2.0
1048463,1048463,16603,Moana,1,2017-06-23,Animation,Moana,2.0


In [ ]:
ratings = pd.DataFrame(df[['user_id_maped','program_id','rating']])

In [ ]:
ratings

,user_id_maped,program_id,rating
0,26138,1.0,1
1,7946,2.0,1
2,7418,3.0,1
3,19307,3.0,2
4,15860,4.0,2
...,...,...,...
1048570,23716,29.0,4
1048571,1776,5042.0,3
1048572,1918,2411.0,1
1048573,25737,420.0,2


In [ ]:
similar_users = ratings[(ratings["program_id"] == movie_id) & (ratings["rating"] >= 4)]["user_id_maped"].unique()
similar_users

array([23732, 32443, 27549, 32439, 14274, 23425, 28506,  1630,  2406,
       27198, 17206, 18800, 12213, 14987,  2015, 26199, 15719, 12251,
        1321, 26769, 20249,  7969, 15637,  8030, 11660, 16327, 20716,
       16558, 32948, 25976, 30851, 18777, 24306, 16680, 17164, 18264,
       18617, 18127,  2120, 33871,  2219, 17700, 33705, 33745, 12041,
       26136,  9664, 24836,  4667, 18083, 15067,  5275, 20737, 15799,
       29829, 26929, 28418,  6009, 29360, 12459, 19326,  1670, 33171,
       10449, 29421,  9317,  4425, 19044, 22325, 13862, 23261, 27892,
       20226,  8228,  4074,  7294,  5094,  8682, 26704,  7519,  6274,
       20807, 33579,  3988, 33135, 12413, 29655,  7134, 15839, 18813,
         671, 28193, 27299,  9761, 25917,  6672, 27032, 14304, 24428,
       27491, 10926, 11132,  5540, 30500, 12171, 11477, 20548, 16901,
       22920,  4049,  6201, 27331,  9765,  3256,  1847, 32490,   343,
        9498, 23299,  5124, 14241, 17610, 24618,  6497, 23118,  9780,
       15965, 20014,

In [ ]:
similar_user_recs = ratings[(ratings["user_id_maped"].isin(similar_users)) & (ratings["rating"] >= 4)]["program_id"]
similar_user_recs

68           51.0
332         182.0
545         260.0
903         394.0
1064         25.0
            ...  
1048341    4077.0
1048465     136.0
1048469     450.0
1048474    1335.0
1048570      29.0
Name: program_id, Length: 21059, dtype: float64

In [ ]:
similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

similar_user_recs = similar_user_recs[similar_user_recs > .10]


In [ ]:
similar_user_recs

2.0       2.850174
53.0      1.585366
24.0      1.104530
709.0     0.933798
603.0     0.459930
            ...   
733.0     0.101045
1642.0    0.101045
3560.0    0.101045
3066.0    0.101045
530.0     0.101045
Name: program_id, Length: 132, dtype: float64

In [ ]:
all_users = ratings[(ratings["program_id"].isin(similar_user_recs.index)) & (ratings["rating"] >= 4)]


In [ ]:
all_users

,user_id_maped,program_id,rating
6,11374,3.0,4
41,18348,33.0,4
52,730,38.0,4
72,18960,55.0,4
125,29524,87.0,4
...,...,...,...
1048515,26231,328.0,4
1048516,23663,2831.0,4
1048533,29320,1283.0,4
1048549,10318,271.0,4


In [ ]:
all_user_recs = all_users["program_id"].value_counts() / len(all_users["user_id_maped"].unique())


In [ ]:
all_user_recs

709.0     0.622432
328.0     0.610307
603.0     0.606938
581.0     0.595150
272.0     0.573931
            ...   
4087.0    0.019198
530.0     0.018525
32.0      0.017514
5287.0    0.013809
565.0     0.013473
Name: program_id, Length: 132, dtype: float64

In [ ]:
rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
rec_percentages.columns = ["similar", "all"]


In [ ]:
rec_percentages

,similar,all
2.0,2.850174,0.275514
53.0,1.585366,0.497474
24.0,1.104530,0.236106
709.0,0.933798,0.622432
603.0,0.459930,0.606938
...,...,...
733.0,0.101045,0.130010
1642.0,0.101045,0.073425
3560.0,0.101045,0.116874
3066.0,0.101045,0.113506


In [ ]:
rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]


In [ ]:
rec_percentages = rec_percentages.sort_values("score", ascending=False)

In [ ]:
rec_percentages.head(10).merge(df, left_index=True, right_on="program_id")


,similar,all,score,Unnamed: 0,user_id_maped,program_name,rating,date_,program_genre,cleaned_title,program_id
1,2.850174,0.275514,10.344948,1,7946,Moana,1,2017-05-21,Animation,Moana,2.0
18,2.850174,0.275514,10.344948,18,26546,Moana,1,2017-06-25,Animation,Moana,2.0
19,2.850174,0.275514,10.344948,19,20306,Moana,1,2017-06-23,Animation,Moana,2.0
48,2.850174,0.275514,10.344948,48,28687,Moana,2,2017-05-30,Animation,Moana,2.0
191,2.850174,0.275514,10.344948,191,16739,Moana,1,2017-05-27,Animation,Moana,2.0
...,...,...,...,...,...,...,...,...,...,...,...
1031550,0.111498,0.020209,5.517305,1031550,262,The pongeBob quarePants Movie,2,2017-11-02,Animation,The pongeBob quarePants Movie,121.0
1031847,0.111498,0.020209,5.517305,1031847,31203,The pongeBob quarePants Movie,2,2017-10-20,Animation,The pongeBob quarePants Movie,121.0
1033551,0.111498,0.020209,5.517305,1033551,22833,The pongeBob quarePants Movie,3,2017-06-04,Animation,The pongeBob quarePants Movie,121.0
1037893,0.111498,0.020209,5.517305,1037893,20720,The pongeBob quarePants Movie,3,2018-02-13,Animation,The pongeBob quarePants Movie,121.0


In [ ]:
def find_similar_movies(movie_id):
    similar_users = ratings[(ratings["program_id"] == movie_id) & (ratings["rating"] > 4)]["user_id_maped"].unique()
    similar_user_recs = ratings[(ratings["user_id_maped"].isin(similar_users)) & (ratings["rating"] > 4)]["program_id"]
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

    similar_user_recs = similar_user_recs[similar_user_recs > .10]
    all_users = ratings[(ratings["program_id"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
    all_user_recs = all_users["program_id"].value_counts() / len(all_users["user_id_maped"].unique())
    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]

    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(df, left_index=True, right_on="program_id")[["score", "program_name", "program_genre"]]


In [ ]:
import ipywidgets as widgets
from IPython.display import display

movie_name_input = widgets.Text(
    disabled=False
)
recommendation_list = widgets.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            results = search(title)
            movie_id = results.iloc[0]["program_id"]
            display(find_similar_movies(movie_id))

movie_name_input.observe(on_type, names='value')

display(movie_name_input, recommendation_list)


Text(value='')

Output()

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_valid=train_test_split(df, test_size=0.1, random_state = 42, stratify=df.rating.values)
# df_train, df_valid=train_test_split(df, test_size=0.1, random_state = 42, stratify=df.rating.value)

# X_train, X_test, y_train, y_test = train_test_split(df, test_size=0.2, random_state = 72)


In [ ]:
!pip install surprise
from surprise import KNNBasic
from surprise.model_selection import train_test_split
from surprise import Dataset, Reader, KNNBasic


In [ ]:
df.groupby('program_name')['rating'].mean().sort_values(ascending=False).head(5)


program_name
#FollowFriday            1.784314
10 Days in a Madhouse    1.707052
100 treets               1.836305
101 Dalmatians           1.466102
102 Dalmatians           1.558140
Name: rating, dtype: float64

In [ ]:
df.groupby('program_name')['rating'].count().sort_values(ascending=False).head()


program_name
The Boss Baby                          24047
Pingu                                  17063
Trolls                                 13793
The Adventures of Petey and Friends     9612
Moana                                   8081
Name: rating, dtype: int64

In [ ]:
ratings = pd.DataFrame(df.groupby('program_name')['rating'].mean())
ratings['num of ratings'] = pd.DataFrame(df.groupby('program_name')['rating'].count())

ratings.head()


,rating,num of ratings
program_name,,
#FollowFriday,1.784314,510
10 Days in a Madhouse,1.707052,553
100 treets,1.836305,1851
101 Dalmatians,1.466102,118
102 Dalmatians,1.558140,172


In [ ]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id_maped', 'program_name', 'rating']], reader)
algo = KNNBasic()
trainset = data.build_full_trainset()
algo.fit(trainset)

# Get recommendations for a user
user_id = 'your_user_id'  # Replace 'your_user_id' with the actual user ID
num_recommendations = 10  # Specify the number of recommendations you want to generate

# Generate recommendations for the user
user_items = df.loc[df['user_id_maped'] == user_id, 'program_name']
user_unrated_items = [item for item in df['program_name'].unique() if item not in user_items]
predictions = [algo.predict(user_id, item) for item in user_unrated_items]
top_recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:num_recommendations]

# Print the top recommendations
print(f"Top {num_recommendations} recommendations for user {user_id}:")
for recommendation in top_recommendations:
    print(f"Program Name: {recommendation.iid}, Estimated Rating: {recommendation.est}")



Computing the msd similarity matrix...
Done computing similarity matrix.
Top 10 recommendations for user your_user_id:
Program Name: 100 treets, Estimated Rating: 2.4972825024437926
Program Name: Moana, Estimated Rating: 2.4972825024437926
Program Name: The Mermaid Princess, Estimated Rating: 2.4972825024437926
Program Name: Churchill, Estimated Rating: 2.4972825024437926
Program Name: Beavis And Butt-Head Do America, Estimated Rating: 2.4972825024437926
Program Name: Coco, Estimated Rating: 2.4972825024437926
Program Name: Kidnap, Estimated Rating: 2.4972825024437926
Program Name: The Accountant, Estimated Rating: 2.4972825024437926
Program Name: The Birth of a Nation, Estimated Rating: 2.4972825024437926
Program Name: The Amazing pider-Man, Estimated Rating: 2.4972825024437926


In [ ]:
df.head(7)

,Unnamed: 0,user_id_maped,program_name,rating,date_,program_genre
0,0,26138,100 treets,1,2017-05-27,Drama
1,1,7946,Moana,1,2017-05-21,Animation
2,2,7418,The Mermaid Princess,1,2017-08-10,Animation
3,3,19307,The Mermaid Princess,2,2017-07-26,Animation
4,4,15860,Churchill,2,2017-07-07,Biography
5,5,20775,Beavis And Butt-Head Do America,1,2017-08-19,Animation
6,6,11374,The Mermaid Princess,4,2017-06-26,Animation


In [ ]:
df.shape

(1048575, 6)

In [ ]:
# df.isnull().any()
# df.duplicated('user_id_maped').sum()
df.duplicated(['user_id_maped', 'program_name','rating']).sum()



453092

In [ ]:
user_scores=df.pivot_table(index=['user_id_maped'], columns=['program_name'], values='rating')
user_scores.head()

program_name,#FollowFriday,10 Days in a Madhouse,100 treets,101 Dalmatians,102 Dalmatians,11.6,12 Monkeys Arms Of Mine,12 Monkeys Atari,12 Monkeys Blood Washed Away,12 Monkeys Bodies Of Water,...,iCarly iLike Jake,iCarly iNevel,iCarly iPilot,iCarly iSpy A Mean Teacher,iCarly iWanna tay With pencer,iCarly iWant A World Record,iCarly iWant More Viewers,iCarly iWill Date Freddie,xXx,xXx: Return of Xander Cage
user_id_maped,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,NaN,1.5,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
user_scores=user_scores.fillna(0)
user_scores=user_scores.replace(np.nan,0)
user_scores.head(10)

program_name,#FollowFriday,10 Days in a Madhouse,100 treets,101 Dalmatians,102 Dalmatians,11.6,12 Monkeys Arms Of Mine,12 Monkeys Atari,12 Monkeys Blood Washed Away,12 Monkeys Bodies Of Water,...,iCarly iLike Jake,iCarly iNevel,iCarly iPilot,iCarly iSpy A Mean Teacher,iCarly iWanna tay With pencer,iCarly iWant A World Record,iCarly iWant More Viewers,iCarly iWill Date Freddie,xXx,xXx: Return of Xander Cage
user_id_maped,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,0.0,1.5,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
user_scores_similarty_df= user_scores.corr(method='pearson')
user_scores_similarty_df.head(10)


program_name,#FollowFriday,10 Days in a Madhouse,100 treets,101 Dalmatians,102 Dalmatians,11.6,12 Monkeys Arms Of Mine,12 Monkeys Atari,12 Monkeys Blood Washed Away,12 Monkeys Bodies Of Water,...,iCarly iLike Jake,iCarly iNevel,iCarly iPilot,iCarly iSpy A Mean Teacher,iCarly iWanna tay With pencer,iCarly iWant A World Record,iCarly iWant More Viewers,iCarly iWill Date Freddie,xXx,xXx: Return of Xander Cage
program_name,,,,,,,,,,,,,,,,,,,,,
#FollowFriday,1.000000,0.114545,0.150652,0.061327,0.101518,0.069378,0.008218,0.010711,0.009705,0.006469,...,0.025131,0.022031,0.042938,0.005494,0.019303,0.010485,0.026084,0.009232,0.105857,0.255334
10 Days in a Madhouse,0.114545,1.000000,0.071788,0.050125,0.058238,0.354388,0.000828,0.006383,0.009276,0.011994,...,0.015130,0.021066,0.015233,0.012551,0.016573,0.017667,0.018111,0.026671,0.077154,0.076398
100 treets,0.150652,0.071788,1.000000,0.053274,0.064619,0.055218,0.008230,0.005361,0.010895,0.014871,...,0.005740,0.004717,0.007730,-0.001359,0.008958,0.000441,0.014927,0.003878,0.073572,0.184588
101 Dalmatians,0.061327,0.050125,0.053274,1.000000,0.308797,0.013285,0.022684,0.017264,-0.003045,-0.003132,...,0.130897,0.110654,0.099244,0.081059,0.099115,0.091598,0.114559,0.102665,0.007826,0.035516
102 Dalmatians,0.101518,0.058238,0.064619,0.308797,1.000000,0.049464,-0.005469,0.001586,-0.003770,-0.003877,...,0.047532,0.037627,0.022690,0.033499,0.018653,0.019705,0.054017,0.026967,0.012347,0.052721
11.6,0.069378,0.354388,0.055218,0.013285,0.049464,1.000000,-0.006812,0.013233,-0.002711,0.001568,...,0.016278,0.012272,0.018678,0.013046,0.013199,0.001084,0.019294,0.003540,0.046854,0.081638
12 Monkeys Arms Of Mine,0.008218,0.000828,0.008230,0.022684,-0.005469,-0.006812,1.000000,0.653842,0.667983,0.634244,...,0.055543,0.031411,0.049107,0.003191,0.018362,0.016348,0.043362,0.042831,0.036639,0.048970
12 Monkeys Atari,0.010711,0.006383,0.005361,0.017264,0.001586,0.013233,0.653842,1.000000,0.448295,0.515687,...,0.007682,0.008483,0.056179,0.015403,0.030279,0.027165,0.042001,0.019243,0.050940,0.060868
12 Monkeys Blood Washed Away,0.009705,0.009276,0.010895,-0.003045,-0.003770,-0.002711,0.667983,0.448295,1.000000,0.840923,...,0.047243,0.043756,0.056288,0.007129,0.024322,0.021947,0.058877,0.057281,0.060160,0.032705
